In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
# Load BigBiGAN module.
module = hub.Module('https://tfhub.dev/deepmind/bigbigan-resnet50/1')



In [ ]:
# Sample a batch of 8 random latent vectors (z) from the Gaussian prior. Then
# call the generator on the latent samples to generate a batch of images with
# shape [8, 128, 128, 3] and range [-1, 1].
z = tf.random.normal([8, 120])  # latent samples
gen_samples = module(z, signature='generate')

# Given a batch of 256x256 RGB images in range [-1, 1], call the encoder to
# compute predicted latents z and other features (e.g. for use in downstream
# recognition tasks).
images = tf.placeholder(tf.float32, shape=[None, 256, 256, 3])
features = module(images, signature='encode', as_dict=True)

# Get the predicted latent sample `z_sample` from the dict of features.
# Other available features include `avepool_feat` and `bn_crelu_feat`, used in
# the representation learning results.
z_sample = features['z_sample']  # shape [?, 120]

# Compute reconstructions of the input `images` by passing the encoder's output
# `z_sample` back through the generator. Note that raw generator outputs are
# half the resolution of encoder inputs (128x128). To get upsampled generator
# outputs matching the encoder input resolution (256x256), instead use:
#     recons = module(z_sample, signature='generate', as_dict=True)['upsampled']
recons = module(z_sample, signature='generate')  # shape [?, 128, 128, 3]